In [1]:
import os, sys
lib_path = os.path.abspath(os.path.join('../..','_Libraries'))
sys.path.append(lib_path)
from rlabs_libutils import DataStruct, select_data, create_outlier_df, find_nearest_above, uniquelist_withidx
from rlabs_liblinreg import method1_useamboutls, method2_splitintrvl # new library for the linear regression functions
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from itertools import izip
#%matplotlib inline

#Read raw data
**Note**: a window will apear with which you can select the data, this window can be hidden by the browser

In [2]:
path = select_data()
ds = DataStruct(path)

Plaid-15.06.11_16.14_jl_nofix_replication_eyetracker_data.txt: eyetracker data (38 colums format)
Trial 1 - 1.5 % of data was lost. SNR: 0.928862926542. CV: 1.07658511436
Trial 2 - 0.7 % of data was lost. SNR: 0.407201998552. CV: 2.45578362473
Trial 3 - 1.3 % of data was lost. SNR: 0.761812301826. CV: 1.31265929626
Trial 4 - 2.2 % of data was lost. SNR: 0.823482607092. CV: 1.21435473122
Trial 5 - 3.1 % of data was lost. SNR: 0.806924052454. CV: 1.23927400225


**Create outlier DataFrame**

In [3]:
outlier_threshold = 100  # velocity values over 100 will be outliers.
ambiguousoutlier_th = 80 # velocity values between 80 and 100 will be ambiguous outliers.
filter_samples = 5       # the samples following an outlier will not be outliers.

df = create_outlier_df(ds,outlier_threshold = outlier_threshold, 
                      ambiguousoutlier_th = ambiguousoutlier_th, filter_samples = filter_samples)

C:\Python27\lib\site-packages\pandas\computation\expressions.py:21: UserWarning: The installed version of numexpr 1.4.2 is not supported in pandas and will be not be used
The minimum supported version is 2.1

  "version is 2.1\n".format(ver=ver), UserWarning)


#Compute linear regression of Gaze position between outliers

In [7]:
outlier_idx = np.where(df['Outlierfiltered'])[0]
n = len(outlier_idx)-1
lr_struct = []
for i in range(n):
    lr_struct.append(regressionbtwpoints(df, outlier_idx[i], outlier_idx[i+1]))

0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
0000000000000000000000000000
00000000000000

In [8]:
outlier_idx = np.where(df['Outlierfiltered'])[0]   											# indexes where are outliers
amb_outlier_idx = np.where(df['isAmbiguousOutlier'])[0] 									# idexes where are ambiguous outliers
n = len(outlier_idx)-1 																		# number of outliers
lr_struct = [] 																				# initialize linear regression list (will be populates with dicts)

for i in range(n): 																			# for each outlier

    fit = regressionbtwpoints(df, outlier_idx[i], outlier_idx[i+1]) 						# compute the linear regression between an outlier and the next outlier
    refineregression(fit, minintervallen = 30, thresrsq = 0.3)

0000000000000000000000000000


NameError: global name 'regressionbtwpoints' is not defined

In [5]:
def refineregression(fit, minintervallen = 30, thresrsq = 0.3, thresslo = 0.0007, fs = 120.0, minsamples = 5):

    if fit['r_squared'] < thresrsq: 														# is the r squared value below r squared threshold?
        nsamples = (df['time'][outlier_idx[i+1]] - df['time'][outlier_idx[i]])/1000.0 * fs 	# if yes, then get bad fit number of samples

        if nsamples > minintervallen:														# is the interval larger than length threshold?

            itvl_start = fit['start'] 														# get interval start
            itvl_end = fit['end'] 															# get interval end

            # can we use ambiguous outliers? -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  
            use_m1 = (np.sum((amb_outlier_idx >= itvl_start+minsamples)\
                &(amb_outlier_idx <= itvl_end-minsamples))) > 0 							# check if there are ambiguous outliers in the interval

            if use_m1: 																		# if yes,
                m1_struct = method1_useamboutls(df, itvl_start, itvl_end, minsamples) 		# use method 1

            # use splitting algorithm (method 2) -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  
            sgmts1, sgmts2, bfidx = method2_splitintrvl(df, itvl_start, itvl_end, maxdivisions, minsamples, fs = 120.0)


            # compare results and use the best -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
            if use_m1:

                cumulative_rsqrd_m1 = np.sum(np.power(np.array([item['r_value'] for item in m1_struct]),2))
                cumulative_rsqrd_m2 = np.sum(np.power(np.array([sgmts1, sgmts2]),2))

                if cumulative_rsqrd_m1 > cumulative_rsqrd_m2:
                    refinedout = m1_struct
                else:
                    refinedout = [sgmts1[bfidx],sgmts2[bfidx]]

            else:
                refinedout = [sgmts1[bfidx],sgmts2[bfidx]]

            #  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -
            out = []
            for item in refinedout:
                newitem = refineregression(item)
                out.append(newitem)

            return out

        else:
            # bad r squared and short interval. Ambiguous
            fit['percept'] = classifyfit(fit, threslen = minintervallen, thresslo = thresslo, thresrsq = thresrsq)
            return fit

    else: 																					# good r squared
        fit['percept'] = classifyfit(fit, threslen = minintervallen, thresslo = thresslo, thresrsq = thresrsq)
        return fit

In [6]:
def regressionbtwpoints(df, start, end, xlabel = 'time', ylabel = 'LEpos_int'):
    """
        Compute the linear regression of df between start and end points.
        
        Input:
        - df: Pandas.DataFrame containing eyetracker data
        - start: starting point for the linear regression
        - end: ending point for the linear regression
        - xlabel: df data to use as horizontal axis of the linear regression. Default as 'time'
        - ylabel: df data to use as vertical axis of the linear regression. Default as 'LEpos_int'

        Output:
        - tmp: dictionary containing linear regression results
        
        References:
        - http://modelling3e4.connectmv.com/wiki/Software_tutorial/Least_squares_modelling_(linear_regression)

    """
    lr_idx = np.arange(start, end)                                                                                  # indices array
    n = len(lr_idx)
    x = df[xlabel][lr_idx]
    y = df[ylabel][lr_idx]

    # compute linear regression:
    slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
    
    # additional calculations:
    r_squared = r_value**2
    resids = y - np.dot(np.vstack([np.ones(n), x]).T, [intercept, slope])       # e = y - Xa; 
    RSS = sum(resids**2) # residual sum of squares
    TSS = sum((y - np.mean(y))**2) # total sum of squares
    R2 = 1 - RSS/TSS
    std_y = np.sqrt(TSS/(n-1)) 

    # add results to temporal struct:
    print '0000000000000000000000000000'
    tmp = {'start':df['time'][start], 'end': df['time'][end], 'start_idx': start, 'end_idx':end,
                 'slope':slope, 'intercept': intercept, 'r_value': r_value, 'p_value': p_value, 'std_err': std_err,
           'r_squared': r_squared, 'resids': resids, 'RSS': RSS, 'TSS': TSS, 'R2': R2, 'std_y': std_y}

    return tmp